In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools
from finbert.custom_finbert import CustomFinBert

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and

In [2]:
standard_dir = project_dir.parent.parent

# ! TODO: PUT THE PREPROCESSING CODE HERE AND STORE PROCESSED DF

# Prepare model

In [4]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
final_model_dir = project_dir/'models'/'classification_model'
data_dir = standard_dir/"Data"
# classification_data_path= standard_dir/'Data'/'sentiment_data'

In [53]:
data = pd.read_csv(standard_dir/"Data"/"processed_data.csv")

In [15]:
train_data = pd.read_csv(standard_dir/"Data"/"train.csv")

In [19]:
train_data.head(4)

,bank_id,description,amount,txn_year,txn_month,txn_day,txn_hour,ATM,Arts and Entertainment,Bank Fees,...,Transfer Deposit,Travel,Utilities,is_interested_investment,is_interested_build_credit,is_interested_increase_income,is_interested_pay_off_debt,is_interested_manage_spending,is_interested_grow_savings,category
0,-1.029810,ACH Withdrawal AMMaryse HemantNEXPRESS TRANSFER,-0.051067,0.0,-1.382342,0.665332,-0.306132,-0.00865,-0.007491,-0.011976,...,-0.065058,-0.022199,-0.0287,-0.149033,-0.094746,-0.143632,-0.11782,-0.119522,-0.109461,Transfer Debit
1,1.146536,CHECKCARD 0819 PANDA EXPRESS 1796 ORLANDO FL 31683168,-0.095158,0.0,0.431279,0.552413,-0.306132,-0.00865,-0.007491,-0.011976,...,-0.065058,-0.022199,-0.0287,-0.149033,-0.094746,-0.143632,-0.11782,-0.119522,-0.109461,Restaurants
2,-0.254545,CKO*Patreon* Member833-9728766 CAUS,-0.054151,0.0,-1.382342,-1.367212,-0.306132,-0.00865,-0.007491,-0.011976,...,-0.065058,-0.022199,-0.0287,-0.149033,-0.094746,-0.143632,-0.11782,-0.119522,-0.109461,Digital Entertainment
3,1.999639,Maryse Hemant,-0.104015,0.0,0.431279,-0.576778,-0.306132,-0.00865,-0.007491,-0.011976,...,-0.065058,-0.022199,-0.0287,-0.149033,-0.094746,-0.143632,-0.11782,-0.119522,-0.109461,Gas Stations


In [22]:
eval_data = pd.read_csv(standard_dir/"Data"/"eval.csv")["category"]
# valid_data = pd.read_csv(standard_dir/"Data"/"validation.csv")

## Configure training parameters

In [5]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at c:\Programming\MoneyLion\standard\advanced\finBERT_transaction_classification\models\language_model\finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [6]:
custom_finbert = CustomFinBert(bertmodel, 38, 30)

In [7]:
config = Config(
    data_dir=data_dir,
    model_dir=final_model_dir,
    bert_model=custom_finbert,
    num_train_epochs=5,
    max_seq_length=64,
    train_batch_size=32,
    learning_rate=2e-5,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=True,
    numeric_params=38,
    label_colname="category"
)

In [8]:
finbert = FinBert(config, transaction_classification=True)

In [10]:
y = pd.read_csv(standard_dir/"Data"/"eval.csv")["category"]

In [11]:
finbert.prepare_model(label_list=y.unique().tolist())

07/17/2024 13:10:50 - INFO - finbert.finbert -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False


In [12]:
train_examples, numeric_feats = finbert.get_data("train")

In [13]:
model = finbert.create_the_model()

In [14]:
trained_model = finbert.train(train_examples = train_examples, model = model, train_numeric_feats=numeric_feats)

07/17/2024 13:13:25 - INFO - finbert.utils -   *** Example ***
07/17/2024 13:13:25 - INFO - finbert.utils -   guid: 0
07/17/2024 13:13:25 - INFO - finbert.utils -   tokens: [CLS] ac ##h withdrawal am ##mar ##yse hem ##ant ##ne ##x ##press transfer [SEP]
07/17/2024 13:13:25 - INFO - finbert.utils -   input_ids: 101 9353 2232 10534 2572 7849 23274 19610 4630 2638 2595 20110 4651 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   label: Transfer Debit (id = 2)
07/17/2024 13:13:44 - INFO - finbert.finbert -   ***** L

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

(Iteration 0) Training loss: 3.2839462757110596


Epoch:   0%|          | 0/5 [00:24<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
trained_model

---

In [131]:
res = bertmodel(input_ids, attention_mask, token_type_ids)

In [132]:
res

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7864, -0.1074,  1.0071,  ...,  0.8979, -0.5206,  0.7588],
        [-0.9847, -0.0000,  1.0389,  ...,  0.0000, -0.5350,  1.0027],
        [-0.7082,  0.0169,  1.0642,  ...,  0.9372, -0.0000,  0.5825],
        ...,
        [-0.8650, -0.1094,  0.9831,  ...,  0.8321, -0.5435,  0.8864],
        [-0.7974, -0.1497,  1.0193,  ...,  0.8952, -0.5464,  0.7718],
        [-0.8178, -0.1569,  0.9880,  ...,  0.8909, -0.4606,  0.7947]]), hidden_states=None, attentions=None)

In [134]:
res[0].shape

torch.Size([32, 768])

In [165]:
X[[col for col in X.columns if col!= "description"]].values.astype(float)

array([[ 1.000e+00,  2.000e+01,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  2.500e+01,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  2.000e+01,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       ...,
       [ 4.880e+02, -1.060e+00,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 4.880e+02, -5.380e-01,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 4.880e+02, -3.996e+00,  2.023e+03, ...,  0.000e+00,  0.000e+00,
         0.000e+00]])

In [168]:
torch.from_numpy(X[[col for col in X.columns if col!= "description"]].values.astype(float))

tensor([[ 1.0000e+00,  2.0000e+01,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  2.5000e+01,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  2.0000e+01,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 4.8800e+02, -1.0600e+00,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 4.8800e+02, -5.3800e-01,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 4.8800e+02, -3.9960e+00,  2.0230e+03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], dtype=torch.float64)

In [139]:
t = torch.rand([32, 38])

In [144]:
t_in = torch.cat([res[0], t], dim=-1)

In [145]:
t_in.shape

torch.Size([32, 806])

In [135]:
custom_classifier = nn.Linear(768+38, 30)

In [147]:
custom_classifier(t_in).shape

torch.Size([32, 30])

### Fine-tune the model

In [85]:
# Get the training examples
train_data = finbert.get_data('train', label_colname='category')

In [100]:
descriptions = X["description"].tolist()

In [101]:
tokenized_descriptions = [tokenizer.tokenize(d) for d in descriptions]

In [103]:
lens = [len(d) for d in tokenized_descriptions]

In [107]:
lens

[7,
 18,
 27,
 11,
 11,
 6,
 4,
 6,
 4,
 6,
 4,
 7,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 13,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 6,
 4,
 7,
 2,
 6,
 6,
 8,
 5,
 6,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 5,
 6,
 7,
 6,
 7,
 6,
 7,
 7,
 6,
 6,
 5,
 7,
 7,
 7,
 7,
 5,
 5,
 5,
 7,
 6,
 6,
 7,
 6,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,

In [95]:
tokenizer.tokenize()

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [93]:
model = finbert.create_the_model()

In [150]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_aff

In [175]:
l = [1,2,3,4]

In [176]:
a,b,c, *d=l

In [177]:
d

[4]

In [94]:
trained_model = finbert.train(train_examples = train_data, model = model)

07/16/2024 17:26:04 - INFO - finbert.utils -   *** Example ***
07/16/2024 17:26:04 - INFO - finbert.utils -   guid: 0
07/16/2024 17:26:04 - INFO - finbert.utils -   tokens: [CLS] online reference # 01 ##0 ##20 ##3 transfer to money market 05 : 03 ##a # 64 ##10 [SEP]
07/16/2024 17:26:04 - INFO - finbert.utils -   input_ids: 101 3784 4431 1001 5890 2692 11387 2509 4651 2000 2769 3006 5709 1024 6021 2050 1001 4185 10790 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   label: Transfer Debit (id = 27)
07/16/2024 17:26:23 - INFO

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [02:43<?, ?it/s]


KeyboardInterrupt: 

---

In [54]:
base_model = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model, do_lower_case=True)